<a href="https://colab.research.google.com/github/Abhiraj36/Malaria-Diagnosis-Using-CNNs/blob/main/Malaria_Detection_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>